In [4]:
# ============================================================================
# BEL Stock Price Forecasting with Chronos and Technical Indicators
# ============================================================================

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# AutoGluon imports
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

print("📊 BEL Stock Forecasting with Technical Indicators")
print("=" * 60)


📊 BEL Stock Forecasting with Technical Indicators


In [ ]:
# ============================================================================
# 1. DATA LOADING AND PREPARATION
# ============================================================================

def load_and_clean_data():
    """Load BEL stock data and perform initial cleaning"""
    
    # Load the CSV file
    data = pd.read_csv('Data/Quote-CD-USDINR-15-09-2024-to-15-09-2025.csv', skipinitialspace=True)
    
    # Clean column names
    data.columns = data.columns.str.strip()
    
    # Convert date and sort chronologically
    data['DATE'] = pd.to_datetime(data['DATE'], format='%d-%b-%Y')
    data = data.sort_values('DATE').reset_index(drop=True)

    # Clean numeric columns (remove commas and convert to float)
    numeric_cols = ['Volume', 'VALUE', 'No of trades']
    for col in numeric_cols:
        data[col] = data[col].str.replace(',', '').astype(float)
    
    print(f"✅ Data loaded: {len(data)} records")
    print(f"📅 Date range: {data['DATE'].min().date()} to {data['DATE'].max().date()}")
    print(f"💰 Price range: ₹{data['CLOSE'].min():.2f} to ₹{data['CLOSE'].max():.2f}")
    print(f"📊 Columns: {', '.join(data.columns)}")
    print(f"Shape: {data.shape}")
    
    return data

# Load the data
raw_data = load_and_clean_data()
raw_data.head()


ValueError: could not convert string to float: '-'

In [3]:

def calculate_expiry_week(dates: pd.Series) -> pd.Series:
    """
    Calculate if a date falls in F&O expiry week (Indian stock market).
    Expiry is typically the last Thursday of each month.
    
    Args:
        dates: pandas Series of datetime objects
    
    Returns:
        pandas Series of boolean values (True if in expiry week)
    """
    def get_last_thursday(year, month):
        """Get the last Thursday of a given month"""
        # Start from the last day of the month and work backwards
        if month == 12:
            last_day = pd.Timestamp(year + 1, 1, 1) - pd.Timedelta(days=1)
        else:
            last_day = pd.Timestamp(year, month + 1, 1) - pd.Timedelta(days=1)
        
        # Find the last Thursday
        days_back = (last_day.weekday() - 3) % 7
        if days_back == 0 and last_day.weekday() != 3:
            days_back = 7
        last_thursday = last_day - pd.Timedelta(days=days_back)
        
        return last_thursday
    
    expiry_week_flags = []
    
    for date in dates:
        # Get last Thursday of this month
        last_thursday = get_last_thursday(date.year, date.month)
        
        # Define expiry week as 4 days before to 1 day after last Thursday
        start_expiry_week = last_thursday - pd.Timedelta(days=4)
        end_expiry_week = last_thursday + pd.Timedelta(days=1)
        
        is_expiry_week = start_expiry_week <= date <= end_expiry_week
        expiry_week_flags.append(is_expiry_week)
    
    return pd.Series(expiry_week_flags, index=dates.index)


def calculate_stochastic(df, k_window=14, d_window=3):
    """
    Calculate Stochastic Oscillator (%K and %D).
    
    %K = (Current Close - Lowest Low) / (Highest High - Lowest Low) * 100
    %D = Simple Moving Average of %K
    
    Args:
        df: DataFrame with 'HIGH', 'LOW', 'close' columns
        k_window: Period for %K calculation (default 14)
        d_window: Period for %D smoothing (default 3)
    
    Returns:
        tuple: (stoch_k, stoch_d) as pandas Series
    """
    # Calculate rolling minimum of lows and maximum of highs
    lowest_low = df['LOW'].rolling(window=k_window, min_periods=1).min()
    highest_high = df['HIGH'].rolling(window=k_window, min_periods=1).max()
    
    # Calculate %K
    stoch_k = 100 * ((df['close'] - lowest_low) / (highest_high - lowest_low))
    
    # Calculate %D (smoothed %K)
    stoch_d = stoch_k.rolling(window=d_window, min_periods=1).mean()
    
    return stoch_k, stoch_d


def calculate_obv(df):
    """
    Calculate On Balance Volume (OBV).
    
    OBV adds volume on up days and subtracts volume on down days.
    It's used to confirm price trends with volume flow.
    
    Args:
        df: DataFrame with 'close' and 'VOLUME' columns
    
    Returns:
        pandas Series: OBV values
    """
    obv = [0]  # Start with 0
    close_prices = df['close'].values
    volumes = df['VOLUME'].values
    
    for i in range(1, len(df)):
        if close_prices[i] > close_prices[i-1]:
            # Price went up, add volume
            obv.append(obv[-1] + volumes[i])
        elif close_prices[i] < close_prices[i-1]:
            # Price went down, subtract volume
            obv.append(obv[-1] - volumes[i])
        else:
            # Price unchanged, OBV unchanged
            obv.append(obv[-1])
    
    return pd.Series(obv, index=df.index)


def calculate_williams_r(df, lookback=14):
    """
    Calculate Williams %R oscillator.
    
    Williams %R = (Highest High - Close) / (Highest High - Lowest Low) * -100
    
    Values range from 0 to -100:
    - Above -20: Overbought
    - Below -80: Oversold
    
    Args:
        df: DataFrame with 'HIGH', 'LOW', 'close' columns
        lookback: Period for calculation (default 14)
    
    Returns:
        pandas Series: Williams %R values
    """
    # Calculate rolling highs and lows
    highest_high = df['HIGH'].rolling(window=lookback, min_periods=1).max()
    lowest_low = df['LOW'].rolling(window=lookback, min_periods=1).min()
    
    # Calculate Williams %R
    williams_r = -100 * ((highest_high - df['close']) / (highest_high - lowest_low))
    
    return williams_r

In [4]:
def calculate_technical_indicators(df):
    """Calculate comprehensive technical indicators for stock analysis - FIXED"""
    
    data = df.copy()
    
    # Simple Moving Averages
    data['SMA_5'] = data['close'].rolling(window=5).mean()
    data['SMA_10'] = data['close'].rolling(window=10).mean()
    data['SMA_20'] = data['close'].rolling(window=20).mean()
    data['SMA_50'] = data['close'].rolling(window=50).mean()
    data['SMA_200'] = data['close'].rolling(window=200).mean()  # Long-term trend
    
    # Exponential Moving Averages
    data['EMA_12'] = data['close'].ewm(span=12).mean()
    data['EMA_26'] = data['close'].ewm(span=26).mean()
    
    # FIXED: Temporal features
    data['day_of_week'] = data['Date'].dt.dayofweek
    data['day_of_month'] = data['Date'].dt.day
    data['month'] = data['Date'].dt.month
    data['quarter'] = data['Date'].dt.quarter
    
    # Market-specific patterns
    data['is_month_end'] = (data['Date'].dt.day > 25).astype(int)
    data['is_expiry_week'] = calculate_expiry_week(data['Date']).astype(int)  # FIXED: Convert to int
    
    # Price vs long-term trend
    data['price_vs_200sma'] = (data['close'] / data['SMA_200'] - 1) * 100  # FIXED: Multiply by 100 for percentage
    
    # FIXED: Momentum oscillators - properly unpack tuples
    stoch_k, stoch_d = calculate_stochastic(data)
    data['stoch_k'] = stoch_k
    data['stoch_d'] = stoch_d
    data['williams_r'] = calculate_williams_r(data)
    
    # FIXED: Volume-price analysis
    data['obv'] = calculate_obv(data)
    data['volume_ma_50'] = data['VOLUME'].rolling(window=50).mean()
    data['volume_breakout'] = (data['VOLUME'] > data['volume_ma_50'] * 1.5).astype(int)
    
    # RSI (Relative Strength Index)
    def calculate_rsi(prices, window=14):
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))
    
    data['RSI'] = calculate_rsi(data['close'])
    data['RSI_7'] = calculate_rsi(data['close'], window=7)   # Short-term
    data['RSI_21'] = calculate_rsi(data['close'], window=21) # Long-term
    
    # MACD (Moving Average Convergence Divergence)
    data['MACD'] = data['EMA_12'] - data['EMA_26']
    data['MACD_signal'] = data['MACD'].ewm(span=9).mean()
    data['MACD_histogram'] = data['MACD'] - data['MACD_signal']
    
    # Bollinger Bands
    data['BB_middle'] = data['close'].rolling(window=20).mean()
    bb_std = data['close'].rolling(window=20).std()
    data['BB_upper'] = data['BB_middle'] + (bb_std * 2)
    data['BB_lower'] = data['BB_middle'] - (bb_std * 2)
    data['BB_width'] = data['BB_upper'] - data['BB_lower']
    data['BB_position'] = (data['close'] - data['BB_lower']) / (data['BB_upper'] - data['BB_lower'])
    
    # Price-based indicators
    data['price_change'] = data['close'].pct_change()
    data['high_low_pct'] = (data['HIGH'] - data['LOW']) / data['close']
    data['open_close_pct'] = (data['close'] - data['OPEN']) / data['OPEN']
    
    # Volume indicators
    data['volume_sma'] = data['VOLUME'].rolling(window=20).mean()
    data['volume_ratio'] = data['VOLUME'] / data['volume_sma']
    data['price_volume'] = data['close'] * data['VOLUME']
    
    # Volatility indicators
    data['volatility'] = data['close'].rolling(window=20).std()
    data['atr'] = ((data['HIGH'] - data['LOW']).rolling(window=14).mean())  # Simplified ATR
    
    # Momentum indicators
    data['momentum'] = data['close'] / data['close'].shift(10) - 1
    data['roc'] = data['close'].pct_change(periods=12)  # Rate of Change
    
    # Support/Resistance levels
    data['support'] = data['LOW'].rolling(window=20).min()
    data['resistance'] = data['HIGH'].rolling(window=20).max()
    data['support_distance'] = (data['close'] - data['support']) / data['close']
    data['resistance_distance'] = (data['resistance'] - data['close']) / data['close']
    
    print("🔧 Enhanced Technical indicators calculated:")
    print("   📈 Moving Averages: SMA(5,10,20,50,200), EMA(12,26)")
    print("   📊 Oscillators: RSI(7,14,21), MACD, Stochastic, Williams %R")
    print("   🎯 Bollinger Bands & Position")
    print("   📉 Volatility: BB Width, ATR, Volatility")
    print("   🚀 Momentum: ROC, Momentum")
    print("   📦 Volume: OBV, Volume Ratio, Price-Volume")
    print("   ⚖️ Support/Resistance levels")
    print("   📅 Temporal: Day/Month effects, F&O expiry")
    
    return data


# Calculate indicators
data_with_indicators = calculate_technical_indicators(raw_data)
print(f"\n📋 Dataset shape: {data_with_indicators.shape}")


🔧 Enhanced Technical indicators calculated:
   📈 Moving Averages: SMA(5,10,20,50,200), EMA(12,26)
   📊 Oscillators: RSI(7,14,21), MACD, Stochastic, Williams %R
   🎯 Bollinger Bands & Position
   📉 Volatility: BB Width, ATR, Volatility
   🚀 Momentum: ROC, Momentum
   📦 Volume: OBV, Volume Ratio, Price-Volume
   ⚖️ Support/Resistance levels
   📅 Temporal: Day/Month effects, F&O expiry

📋 Dataset shape: (248, 59)


In [5]:
# ============================================================================
# 3. FEATURE SELECTION AND DATA PREPARATION
# ============================================================================

def prepare_features_for_modeling(df, target_col='close'):
    """Select and prepare features for time series modeling"""
    
    # Select the most relevant features for forecasting
    feature_columns = [
        'Date', target_col,
        
        # **Short-term momentum indicators** (most important for 3-day)
        'RSI', 'RSI_7',                    # Momentum signals
        'stoch_k', 'stoch_d',              # Overbought/oversold
        'williams_r',                       # Momentum confirmation
        'MACD', 'MACD_signal',             # Trend changes
        
        # **Price action** 
        'SMA_5', 'SMA_10', 'SMA_20',      # Short-term trends
        'EMA_12', 'EMA_26',               # Responsive averages
        'high_low_pct', 'open_close_pct', # Daily volatility
        
        # **Volume confirmation**
        'obv',                             # Volume flow
        'volume_ratio',                    # Volume spikes
        'volume_breakout',                 # Unusual activity
        
        # **Bollinger Bands** (for volatility regime)
        'BB_position', 'BB_width',
        
        # **Market structure**
        'support_distance', 'resistance_distance',
        'volatility', 'momentum',
        
        # **Temporal patterns** (crucial for 3-day)
        'day_of_week', 'is_expiry_week',   # Weekly patterns
        'is_month_end'                     # Monthly patterns
    ]
    
    # Create modeling dataset
    model_data = df[feature_columns].copy()
    
    # Drop rows with NaN values (from rolling calculations)
    model_data = model_data.dropna().reset_index(drop=True)
    
    # Prepare for AutoGluon TimeSeriesDataFrame
    ts_data = model_data.rename(columns={'Date': 'timestamp', target_col: 'target'})
    ts_data['item_id'] = 'BEL'
    
    # Reorder columns: item_id, timestamp, target, covariates...
    covariate_cols = [col for col in ts_data.columns if col not in ['item_id', 'timestamp', 'target']]
    column_order = ['item_id', 'timestamp', 'target'] + covariate_cols
    ts_data = ts_data[column_order]
    
    print(f"🎯 Features selected for modeling: {len(covariate_cols)} covariates")
    print(f"📊 Clean dataset: {len(ts_data)} records (removed {len(df) - len(ts_data)} NaN rows)")
    print(f"📋 Covariates: {covariate_cols}")
    
    return ts_data, covariate_cols

# Prepare features
ts_data, covariate_names = prepare_features_for_modeling(data_with_indicators)
ts_data.head()


🎯 Features selected for modeling: 26 covariates
📊 Clean dataset: 229 records (removed 19 NaN rows)
📋 Covariates: ['RSI', 'RSI_7', 'stoch_k', 'stoch_d', 'williams_r', 'MACD', 'MACD_signal', 'SMA_5', 'SMA_10', 'SMA_20', 'EMA_12', 'EMA_26', 'high_low_pct', 'open_close_pct', 'obv', 'volume_ratio', 'volume_breakout', 'BB_position', 'BB_width', 'support_distance', 'resistance_distance', 'volatility', 'momentum', 'day_of_week', 'is_expiry_week', 'is_month_end']


,item_id,timestamp,target,RSI,RSI_7,stoch_k,stoch_d,williams_r,MACD,MACD_signal,...,volume_breakout,BB_position,BB_width,support_distance,resistance_distance,volatility,momentum,day_of_week,is_expiry_week,is_month_end
0,BEL,2024-10-14,285.70,49.481865,60.903427,69.000000,70.555556,-31.000000,-0.117059,-0.453722,...,0,0.577729,26.405779,0.072454,0.032552,6.601445,-0.026410,0,0,0
1,BEL,2024-10-15,288.85,47.344734,67.259259,79.500000,72.722222,-20.500000,0.267773,-0.308080,...,0,0.702247,26.106696,0.082569,0.021291,6.526674,0.013153,1,0,0
2,BEL,2024-10-16,285.70,46.343612,83.918669,69.346734,72.615578,-30.653266,0.351232,-0.175237,...,0,0.578717,26.169617,0.072454,0.032552,6.542404,0.006163,2,0,0
3,BEL,2024-10-17,284.55,44.803493,64.052288,65.494137,71.446957,-34.505863,0.333666,-0.072852,...,0,0.531524,26.170574,0.068705,0.036725,6.542644,0.020990,3,0,0
4,BEL,2024-10-18,287.15,44.463972,65.079365,74.831081,69.890651,-25.168919,0.494827,0.041222,...,0,0.611866,24.158379,0.077137,0.027338,6.039595,0.035895,4,0,0


In [6]:
# ============================================================================
# 4. TRAIN-TEST SPLIT WITH FREQUENCY HANDLING
# ============================================================================

def create_train_test_split_with_covariates(ts_data, covariate_names, test_days=30):
    """Create train-test split with proper frequency and covariate handling"""
    
    # Convert to TimeSeriesDataFrame with covariates
    ts_df = TimeSeriesDataFrame.from_data_frame(
        ts_data,
        id_column='item_id',
        timestamp_column='timestamp'
    )
    
    # Set frequency to daily
    ts_df = ts_df.convert_frequency(freq='D')
    
    # Train-test split based on time
    last_date = ts_df.index.get_level_values('timestamp').max()
    split_date = last_date - pd.Timedelta(days=test_days)
    
    train_data = ts_df.loc[ts_df.index.get_level_values('timestamp') <= split_date]
    test_data = ts_df.loc[ts_df.index.get_level_values('timestamp') > split_date]
    
    # Ensure frequency is maintained
    train_data = train_data.convert_frequency(freq='D')
    test_data = test_data.convert_frequency(freq='D')
    
    print("🔄 Train-Test Split:")
    print(f"   📈 Training data: {len(train_data)} points (until {split_date.date()})")
    print(f"   🔍 Test data: {len(test_data)} points")
    print(f"   📊 Frequency: {train_data.freq}")
    print(f"   🎯 Covariates: {len(covariate_names)} features")
    
    return train_data, test_data, split_date, ts_df

# Create train-test split
train_data, test_data, split_date, full_ts_df = create_train_test_split_with_covariates(
    ts_data, covariate_names, test_days=10
)


🔄 Train-Test Split:
   📈 Training data: 324 points (until 2025-09-02)
   🔍 Test data: 10 points
   📊 Frequency: D
   🎯 Covariates: 26 features


In [13]:
# ============================================================================
# 5. CHRONOS MODEL TRAINING (BASE MODEL ONLY)
# ============================================================================

def train_chronos_base_model(train_data, test_data, covariate_names):
    """Train Chronos-Bolt-Base model with technical indicators"""
    
    prediction_length = len(test_data)
    
    print("🚀 Training Chronos-Bolt-Base Model")
    print(f"   📏 Prediction length: {prediction_length} days")
    print(f"   🎯 Using {len(covariate_names)} covariates")
    print("-" * 50)
    
    try:
        # Initialize predictor with explicit frequency and covariates
        predictor = TimeSeriesPredictor(
            target='target',
            prediction_length=prediction_length,
            path="./chronos_models/bel_bolt_base_with_indicators",
            freq='D',  # Daily frequency
            verbosity=2
        )
        
        # Train the model
        predictor.fit(
            train_data=train_data,
            hyperparameters={
                "Chronos": [
                    {
                        "model_path": "bolt_base",
                        "ag_args": {"name_suffix": "ZeroShot"}
                    }
                ]
            },
            presets='bolt_base',  # Use bolt_base only
            time_limit=600,  # 10 minutes max
            skip_model_selection=False  # Use only the specified preset
        )
        
        # Make predictions
        predictions = predictor.predict(train_data)
        
        print("✅ Chronos-Bolt-Base training completed successfully!")
        
        return predictor, predictions
        
    except Exception as e:
        print(f"❌ Model training failed: {str(e)}")
        return None, None

# Train the model
predictor, predictions = train_chronos_base_model(train_data, test_data, covariate_names)


Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to '/workspaces/Market/chronos_models/bel_bolt_base_with_indicators'
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #29~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Aug 14 16:52:50 UTC 2
CPU Count:          8
GPU Count:          1
Memory Avail:       12.17 GB / 23.31 GB (52.2%)
Disk Space Avail:   357.68 GB / 931.51 GB (38.4%)
Setting presets to: bolt_base

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'D',
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'ZeroShot'},
                                  'model_path': 'bolt_base'}]},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 10,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': Fa

🚀 Training Chronos-Bolt-Base Model
   📏 Prediction length: 10 days
   🎯 Using 26 covariates
--------------------------------------------------


	-0.0135       = Validation score (-WQL)
	0.02    s     = Training runtime
	2.02    s     = Validation (prediction) runtime
Not fitting ensemble as only 1 model was trained.
Training complete. Models trained: ['ChronosZeroShot[bolt_base]']
Total runtime: 2.06 s
Best model: ChronosZeroShot[bolt_base]
Best model score: -0.0135
Model not specified in predict, will default to the model with the best validation score: ChronosZeroShot[bolt_base]


✅ Chronos-Bolt-Base training completed successfully!


In [14]:
# ============================================================================
# 6. FIXED MODEL EVALUATION
# ============================================================================

def evaluate_predictions(predictions, test_data, ts_data):
    """Evaluate model performance with detailed metrics - FIXED VERSION"""
    
    if predictions is None:
        print("❌ No predictions available for evaluation")
        return None
    
    # Get actual test values
    test_dates = test_data.index.get_level_values('timestamp')
    actual_values = test_data['target'].values
    
    # Get predicted values - FIXED
    pred_df = predictions.reset_index()
    
    # Handle different column names in predictions
    if 'mean' in pred_df.columns:
        predicted_values = pred_df['mean'].values[:len(actual_values)]
    elif '0.5' in pred_df.columns:  # Median prediction
        predicted_values = pred_df['0.5'].values[:len(actual_values)]
    elif 'target' in pred_df.columns:
        predicted_values = pred_df['target'].values[:len(actual_values)]
    else:
        print(f"⚠️ Available prediction columns: {pred_df.columns.tolist()}")
        predicted_values = pred_df.iloc[:, -1].values[:len(actual_values)]  # Use last column
    
    # Remove NaN values if present
    valid_mask = ~(np.isnan(actual_values) | np.isnan(predicted_values))
    actual_values_clean = actual_values[valid_mask]
    predicted_values_clean = predicted_values[valid_mask]
    test_dates_clean = test_dates[valid_mask]
    
    if len(actual_values_clean) == 0:
        print("❌ No valid prediction-actual pairs found")
        return None
    
    # Calculate metrics
    mae = np.mean(np.abs(actual_values_clean - predicted_values_clean))
    rmse = np.sqrt(np.mean((actual_values_clean - predicted_values_clean) ** 2))
    mape = np.mean(np.abs((actual_values_clean - predicted_values_clean) / actual_values_clean)) * 100
    
    # Direction accuracy (if price goes up/down correctly predicted)
    if len(actual_values_clean) > 1:
        actual_direction = np.diff(actual_values_clean) > 0
        pred_direction = np.diff(predicted_values_clean) > 0
        direction_accuracy = np.mean(actual_direction == pred_direction) * 100
    else:
        direction_accuracy = 0.0
    
    print("📊 Model Performance Evaluation:")
    print(f"   📏 MAE:  ₹{mae:.2f}")
    print(f"   📐 RMSE: ₹{rmse:.2f}")
    print(f"   📊 MAPE: {mape:.2f}%")
    print(f"   🎯 Direction Accuracy: {direction_accuracy:.1f}%")
    print(f"   ✅ Valid predictions: {len(actual_values_clean)}/{len(actual_values)}")
    
    # Create evaluation results
    results = {
        'actual_values': actual_values_clean,
        'predicted_values': predicted_values_clean,
        'test_dates': test_dates_clean,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'direction_accuracy': direction_accuracy
    }
    
    return results

# Re-evaluate the model with fixed function
eval_results = evaluate_predictions(predictions, test_data, ts_data)


📊 Model Performance Evaluation:
   📏 MAE:  ₹14.10
   📐 RMSE: ₹18.12
   📊 MAPE: 3.64%
   🎯 Direction Accuracy: 57.1%
   ✅ Valid predictions: 8/10


In [15]:
# ============================================================================
# 7. FIXED COMPREHENSIVE VISUALIZATION
# ============================================================================

def create_comprehensive_forecast_visualization(ts_data, eval_results, split_date):
    """Create detailed interactive visualization - FIXED VERSION"""
    
    if eval_results is None:
        print("❌ No evaluation results available for visualization")
        return None
    
    # Convert split_date to Python datetime to fix Plotly compatibility
    if hasattr(split_date, 'to_pydatetime'):
        split_date_fixed = split_date.to_pydatetime()
    else:
        split_date_fixed = split_date
    
    # Prepare data for plotting
    if hasattr(ts_data, 'reset_index'):
        full_data = ts_data.reset_index()
    else:
        full_data = ts_data.copy()
    
    train_data_plot = full_data[full_data['timestamp'] <= split_date]
    test_data_plot = full_data[full_data['timestamp'] > split_date]
    
    # Create subplots
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        subplot_titles=(
            'Price Forecast vs Actual', 
            'Prediction Error Analysis',
            'Technical Indicators'
        ),
        row_heights=[0.5, 0.25, 0.25]
    )
    
    # 1. Main price chart
    # Historical prices
    fig.add_trace(
        go.Scatter(
            x=train_data_plot['timestamp'],
            y=train_data_plot['target'],
            name='Historical Prices',
            line=dict(color='blue', width=2),
            mode='lines'
        ),
        row=1, col=1
    )
    
    # Actual test prices
    fig.add_trace(
        go.Scatter(
            x=eval_results['test_dates'],
            y=eval_results['actual_values'],
            name='Actual Prices',
            line=dict(color='white', width=3),
            mode='lines+markers',
            marker=dict(size=4)
        ),
        row=1, col=1
    )
    
    # Predicted prices
    fig.add_trace(
        go.Scatter(
            x=eval_results['test_dates'],
            y=eval_results['predicted_values'],
            name=f'Chronos Forecast (RMSE: ₹{eval_results["rmse"]:.2f})',
            line=dict(color='red', width=2, dash='dash'),
            mode='lines+markers',
            marker=dict(size=4)
        ),
        row=1, col=1
    )
    
    # 2. Error analysis
    errors = eval_results['actual_values'] - eval_results['predicted_values']
    fig.add_trace(
        go.Scatter(
            x=eval_results['test_dates'],
            y=errors,
            name='Prediction Error (₹)',
            line=dict(color='purple', width=2),
            mode='lines+markers',
            fill='tonexty'
        ),
        row=2, col=1
    )
    
    # Add zero line for errors
    fig.add_hline(y=0, line_dash="dot", line_color="gray", row=2, col=1)
    
    # 3. Technical indicators (RSI as example)
    if 'RSI' in full_data.columns:
        rsi_data = full_data['RSI'].dropna()
        rsi_dates = full_data.loc[rsi_data.index, 'timestamp']
        
        fig.add_trace(
            go.Scatter(
                x=rsi_dates,
                y=rsi_data,
                name='RSI',
                line=dict(color='orange', width=1),
                mode='lines'
            ),
            row=3, col=1
        )
        
        # RSI reference lines
        fig.add_hline(y=70, line_dash="dash", line_color="red", opacity=0.5, row=3, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", opacity=0.5, row=3, col=1)
    
    # FIXED: Add split line with datetime conversion
    fig.add_vline(
        x=split_date_fixed,  # Use the fixed datetime
        line_dash="dot",
        line_color="gray"
    )
    
    # Add annotation separately to avoid the timestamp addition issue
    fig.add_annotation(
        x=split_date_fixed,
        y=max(train_data_plot['target']) * 0.95,
        text="Train/Test Split",
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="gray"
    )
    
    # Update layout
    fig.update_layout(
        title='BEL Stock Price Forecasting with Chronos-Bolt-Base<br>' +
              f'<sub>MAPE: {eval_results["mape"]:.2f}% | Direction Accuracy: {eval_results["direction_accuracy"]:.1f}%</sub>',
        height=900,
        template='plotly_dark',
        hovermode='x unified',
        showlegend=True
    )
    
    # Update axes
    fig.update_yaxes(title_text="Price (₹)", row=1, col=1)
    fig.update_yaxes(title_text="Error (₹)", row=2, col=1)
    fig.update_yaxes(title_text="RSI", row=3, col=1)
    fig.update_xaxes(title_text="Date", row=3, col=1)
    
    return fig

# Create comprehensive visualization with fixed function
forecast_fig = create_comprehensive_forecast_visualization(ts_data, eval_results, split_date)

if forecast_fig:
    forecast_fig.show()
    
    # Save the chart
    forecast_fig.write_html("bel_chronos_forecast_with_indicators.html")
    print("💾 Forecast visualization saved as 'bel_chronos_forecast_with_indicators.html'")
else:
    print("⚠️ Visualization could not be created")


💾 Forecast visualization saved as 'bel_chronos_forecast_with_indicators.html'
